In [5]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.model_selection import train_test_split
# Building a english to french translator

In [6]:
# Get the data
names=pd.read_csv('human_names.csv')

In [7]:
names.name=names.name.apply(lambda x: x.lower())

In [8]:
names=pd.DataFrame({'name':names.name.unique()})

In [9]:
names.head()

,name
0,john
1,william
2,james
3,charles
4,george


In [30]:
lenght_list=[]
for l in names.name:
    lenght_list.append(len(l))
max_len = np.max(lenght_list)+1

In [31]:
# Append '\t' to input to delay target by 1 timstamp
names['name']=names.name.apply(lambda x:'\t'+x)

In [32]:
# Append \n to notify end of word
names['target']=names.name.apply(lambda x:x[1:len(x)]+'\n')

In [33]:
names.shape

(6782, 2)

In [34]:
# Get the vocab dict
all_chars=set()
for name in names.name:
    for c in name:
        if c not in all_chars:
            all_chars.add(c)
all_chars.add('\n')

# max length of a name is 11
char_to_ix = { ch:i for i,ch in enumerate(sorted(all_chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(all_chars)) }

In [35]:
input_data = np.zeros((len(names.name), max_len, 28),dtype='float32')
output_data = np.zeros((len(names.name), max_len, 28),dtype='float32')

In [36]:
# Generate input and output data
for i, x in enumerate(names.name):
    for t, ch in enumerate(x):
        input_data[i, t, char_to_ix[ch]] = 1.
for i, x in enumerate(names.target):
    for t, ch in enumerate(x):
        output_data[i,t, char_to_ix[ch]] = 1.

In [38]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import LSTM, Dropout
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import plot_model

In [39]:
model = Sequential()
model.add(LSTM(50, input_shape=(max_len, len(all_chars)), return_sequences=True))
model.add(TimeDistributed(Dense(len(all_chars))))
model.add(TimeDistributed(Activation('softmax')))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [40]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 13, 50)            15800     
_________________________________________________________________
time_distributed (TimeDistri (None, 13, 28)            1428      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 13, 28)            0         
Total params: 17,228
Trainable params: 17,228
Non-trainable params: 0
_________________________________________________________________


In [41]:
# Sampling function to generate new names
def onend(epoch, logs):
    if epoch%2==0 and epoch !=0:
        print('----- Generating text after Epoch: %d' % epoch)
        for i in range(0,10):
            stop=False
            ch='\t'
            counter=1
            target_seq = np.zeros((1, max_len, 28))
            target_seq[0, 0, char_to_ix[ch]] = 1.
            while stop == False and counter < 10:
                #sample the data
                probs = model.predict_proba(target_seq, verbose=0)[:,counter-1,:]
                c= np.random.choice(sorted(list(all_chars)), replace =False,p=probs.reshape(28))
                #c=ix_to_char[np.argmax(probs.reshape(28))]
                if c=='\n':
                    stop=True
                else:
                    ch=ch+c
                    target_seq[0,counter , char_to_ix[c]] = 1.
                    counter=counter+1
            print(ch)


In [43]:
# fit the model
print_callback = LambdaCallback(on_epoch_end=onend)
model.fit(input_data, output_data, batch_size=32,epochs=100, callbacks=[print_callback])

Train on 6782 samples
Epoch 1/100
6782/6782 [==============================] - 1s 99us/sample - loss: 1.0827
Epoch 2/100
6782/6782 [==============================] - 1s 116us/sample - loss: 1.0620
Epoch 3/100
6528/6782 [===========================>..] - ETA: 0s - loss: 1.0459----- Generating text after Epoch: 2
		charlie
		bristie
		lel
		sena
		alsie
		eargelin
		frana
		gabenta
		hain
		michurle
6782/6782 [==============================] - 3s 429us/sample - loss: 1.0449
Epoch 4/100
6782/6782 [==============================] - 1s 104us/sample - loss: 1.0303
Epoch 5/100
6592/6782 [============================>.] - ETA: 0s - loss: 1.0176----- Generating text after Epoch: 4
		shaquan
		tol
		katis
		eramor
		emaria
		milphera
		isa
		lolita
		hedring
		adiah
6782/6782 [==============================] - 3s 444us/sample - loss: 1.0184
Epoch 6/100
6782/6782 [==============================] - 1s 106us/sample - loss: 1.0062
Epoch 7/100
6528/6782 [===========================>..] - ETA: 0s - lo

In [44]:
for i in range(0,10):
    stop=False
    ch='\t'
    counter=1
    target_seq = np.zeros((1, max_len, 28))
    target_seq[0, 0, char_to_ix[ch]] = 1.
    while stop == False and counter < 10:
        #sample the data
        probs = model.predict_proba(target_seq, verbose=0)[:,counter-1,:]
        c= np.random.choice(sorted(list(all_chars)), replace =False,p=probs.reshape(28))
        #c=ix_to_char[np.argmax(probs.reshape(28))]
        if c=='\n':
            stop=True
        else:
            ch=ch+c
            target_seq[0,counter , char_to_ix[c]] = 1.
            counter=counter+1
    print(ch)


		sentho
		davis
		tawerl
		nayere
		heylie
		jaylynn
		janell
		vamarcus
		marcelia
		nakory
